In [139]:
from striprtf.striprtf import rtf_to_text
from sklearn.model_selection import TimeSeriesSplit, KFold, StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
import warnings
from sklearn.metrics import mean_squared_error, r2_score
warnings.filterwarnings('ignore')
import json
import pandas as pd
import numpy as np

In [127]:
def convert_rtf_json(rtf_file_path,output_json_path):
    with open(rtf_file_path, "r") as file:
        rtf_content = file.read()
    plain_text = rtf_to_text(rtf_content)
    try:
        json_data = json.loads(plain_text)  
    except json.JSONDecodeError:
        print("Error: The RTF content is not valid JSON. Please clean up the content.")
        raise
    with open(output_json_path, "w") as json_file:
        json.dump(json_data, json_file, indent=4)
    
    print(f"Converted JSON saved to: {output_json_path}")

In [128]:
rtf_file_path = convert_rtf_json("algoparams_from_ui.json.rtf","algoparams_from_ui.json")


Converted JSON saved to: algoparams_from_ui.json


In [129]:
with open("algoparams_from_ui.json",'r') as file:
    data = json.load(file)
print(data)

{'session_name': 'test', 'session_description': 'test', 'design_state_data': {'session_info': {'project_id': '1', 'experiment_id': 'kkkk-11', 'dataset': 'iris_modified.csv', 'session_name': 'test', 'session_description': 'test'}, 'target': {'prediction_type': 'Regression', 'target': 'petal_width', 'type': 'regression', 'partitioning': True}, 'train': {'policy': 'Split the dataset', 'time_variable': 'sepal_length', 'sampling_method': 'No sampling(whole data)', 'split': 'Randomly', 'k_fold': False, 'train_ratio': 0, 'random_seed': 0}, 'metrics': {'optomize_model_hyperparameters_for': 'AUC', 'optimize_threshold_for': 'F1 Score', 'compute_lift_at': 0, 'cost_matrix_gain_for_true_prediction_true_result': 1, 'cost_matrix_gain_for_true_prediction_false_result': 0, 'cost_matrix_gain_for_false_prediction_true_result': 0, 'cost_matrix_gain_for_false_prediction_false_result': 0}, 'feature_handling': {'sepal_length': {'feature_name': 'sepal_length', 'is_selected': True, 'feature_variable_type': 'nu

In [52]:
def extracting_target_predictive_type(json_data):
    target_dic = {}
    target_main = json_data["design_state_data"]["target"]
    target_colum = target_main['target']
    prediction_type = target_main["prediction_type"]
    target_dic ={'target' : target_colum, 'prediction_type': prediction_type}
    print(f'{target_colum}')
    print(f"{prediction_type}")
    return target_dic

In [53]:
target_predictive_type = extracting_target_predictive_type(data)

petal_width
Regression


In [106]:
def load_data(csv_path):
    dataframe = pd.read_csv(csv_path)
    return dataframe

In [99]:
def checking_null(csv_path):
    df = pd.read_csv(csv_path)
    null_value = df.isnull().sum()
    return null_value
    

In [55]:
df =pd.read_csv('iris.csv')

In [11]:
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [13]:
df.isnull().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64

In [138]:
y = df[target_predictive_type['target']]
X = df.drop(target_predictive_type['target'], axis=1)

In [44]:
y

0      0.2
1      0.2
2      0.2
3      0.2
4      0.2
      ... 
145    2.3
146    1.9
147    2.0
148    2.3
149    1.8
Name: petal_width, Length: 150, dtype: float64

In [46]:
X

,sepal_length,sepal_width,petal_length,species
0,5.1,3.5,1.4,Iris-setosa
1,4.9,3.0,1.4,Iris-setosa
2,4.7,3.2,1.3,Iris-setosa
3,4.6,3.1,1.5,Iris-setosa
4,5.0,3.6,1.4,Iris-setosa
...,...,...,...,...
145,6.7,3.0,5.2,Iris-virginica
146,6.3,2.5,5.0,Iris-virginica
147,6.5,3.0,5.2,Iris-virginica
148,6.2,3.4,5.4,Iris-virginica


In [67]:


def encode_categorical_features(X):
    X_encoded = X.copy() 

    for column in X_encoded.select_dtypes(include=['object', 'category']).columns:
        le = LabelEncoder()
        X_encoded[column] = le.fit_transform(X_encoded[column])

    return X_encoded  


In [68]:
encode_feature = encode_categorical_features(X)

In [70]:
encode_feature

,sepal_length,sepal_width,petal_length,species
0,5.1,3.5,1.4,0
1,4.9,3.0,1.4,0
2,4.7,3.2,1.3,0
3,4.6,3.1,1.5,0
4,5.0,3.6,1.4,0
...,...,...,...,...
145,6.7,3.0,5.2,2
146,6.3,2.5,5.0,2
147,6.5,3.0,5.2,2
148,6.2,3.4,5.4,2


In [69]:
print(encode_feature.dtypes)

sepal_length    float64
sepal_width     float64
petal_length    float64
species           int32
dtype: object


In [17]:
def handle_missing_values(df, json_data):
    feature_handling = json_data["design_state_data"]["feature_handling"]
    df_imputed = df.copy()
    
    for feature_name, feature_data in feature_handling.items():
        if not feature_data.get("is_selected") or not df[feature_name].isnull().any():
            continue
            
        impute_with = feature_data["feature_details"].get("impute_with")
        
        if impute_with == "Average of values":
            fill_value = df[feature_name].mean()
            df_imputed[feature_name].fillna(fill_value, inplace=True)
            print(f"Imputed {feature_name} with mean value: {fill_value}")
            
        elif impute_with == "custom":
            fill_value = feature_data["feature_details"].get("impute_value")
            df_imputed[feature_name].fillna(fill_value, inplace=True)
            print(f"Imputed {feature_name} with custom value: {fill_value}")
    
    return df_imputed

In [18]:
mis = handle_missing_values(df,data)

In [19]:
mis

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [75]:
def feature_reduction(X, y, json_data, target_info,X_encoded):
   
    reduction_config = json_data["design_state_data"]["feature_reduction"]
    method = reduction_config["feature_reduction_method"]
    n_features = int(reduction_config["num_of_features_to_keep"])
    
    print(f"Applying {method} feature reduction...")
    
    if method == "No Reduction":
        print("No feature reduction applied")
        return X
        
    elif method == "Corr with Target":
        correlations = {}
        for column in X.columns:
            if X[column].dtype == "object":  
                continue
            correlation = X[column].corr(y)
            correlations[column] = abs(correlation)
            
        selected_features = sorted(correlations.items(), 
                                   key=lambda x: x[1], 
                                   reverse=True)[:n_features]
        selected_columns = [feat[0] for feat in selected_features]
        
        print("Correlation values:")
        for feat, corr in selected_features:
            print(f"{feat}: {corr:.3f}")
            
        print(f"\nSelected features based on correlation: {selected_columns}")
        return X[selected_columns]
        
    elif method == "Tree-based":
        n_trees = int(reduction_config["num_of_trees"])
        max_depth = int(reduction_config["depth_of_trees"])
        
        if target_info["prediction_type"] == "Regression":
            rf_params = json_data["design_state_data"]["algorithms"]["RandomForestRegressor"]
            model = RandomForestRegressor(
                n_estimators=rf_params["max_trees"],
                max_depth=rf_params["max_depth"],
                min_samples_leaf=rf_params["min_samples_per_leaf_min_value"],
                random_state=42
            )
            
        model.fit(X_encoded, y)
        importances = dict(zip(X_encoded.columns, model.feature_importances_))
        selected_features = sorted(importances.items(), 
                                   key=lambda x: x[1], 
                                   reverse=True)[:n_features]
        selected_columns = [feat[0] for feat in selected_features]
        
        print(f"Selected features based on tree importance: {selected_columns}")
        print(X_encoded[selected_columns])
        return X_encoded[selected_columns]
        
    elif method == "PCA":
        pca = PCA(n_components=n_features)
        X_pca = pca.fit_transform(X_encoded)
        pca_columns = [f'PC{i+1}' for i in range(n_features)]
        
        print(f"Variance explained by PCA components: {pca.explained_variance_ratio_}")
        return pd.DataFrame(X_pca, columns=pca_columns, index=X_encoded.index)
        
    else:
        raise ValueError(f"Unknown reduction method: {method}")


In [107]:
reduction= feature_reduction(X,y,data,target_predictive_type,encode_feature)

Applying Tree-based feature reduction...
Selected features based on tree importance: ['species', 'petal_length', 'sepal_width', 'sepal_length']
     species  petal_length  sepal_width  sepal_length
0          0           1.4          3.5           5.1
1          0           1.4          3.0           4.9
2          0           1.3          3.2           4.7
3          0           1.5          3.1           4.6
4          0           1.4          3.6           5.0
..       ...           ...          ...           ...
145        2           5.2          3.0           6.7
146        2           5.0          2.5           6.3
147        2           5.2          3.0           6.5
148        2           5.4          3.4           6.2
149        2           5.1          3.0           5.9

[150 rows x 4 columns]


In [81]:
def select_models(json_data, target_info):

    algorithms = json_data["design_state_data"]["algorithms"]
    prediction_type = target_info['prediction_type']
    selected_models = {}
    
    print(f"\nSelecting models for {prediction_type}")
    
    for model_name, model_config in algorithms.items():
        if model_config["is_selected"]:
            print(f"\nModel {model_name} is selected")
            if prediction_type == "Regression":
                if any(x in model_name for x in ["Regressor", "Regression"]):
                    selected_models[model_name] = model_config
                    print(f"Added {model_name} for regression task")
                else:
                    print(f"Skipping {model_name} - not suitable for regression")
    
    return selected_models


In [115]:
selected_models = select_models(data, target_predictive_type)
print("\nFinal Selected Models with Parameters:")
print("=" * 50)
for model_name, model_config in selected_models.items():
    print(f"\n{model_name}:")
    for param_name, param_value in model_config.items():
        if param_name != "model_name":
            print(f"- {param_name}: {param_value}")


Selecting models for Regression

Model RandomForestRegressor is selected
Added RandomForestRegressor for regression task

Final Selected Models with Parameters:

RandomForestRegressor:
- is_selected: True
- min_trees: 10
- max_trees: 20
- feature_sampling_statergy: Default
- min_depth: 20
- max_depth: 25
- min_samples_per_leaf_min_value: 5
- min_samples_per_leaf_max_value: 10
- parallelism: 0


In [85]:
def parse_hyper(json_data):
     parms = json_data["design_state_data"]["hyperparameters"]
     return parms

In [87]:
parameter = parse_hyper(data)

In [88]:
parameter

{'stratergy': 'Grid Search',
 'shuffle_grid': True,
 'random_state': 1,
 'max_iterations': 2,
 'max_search_time': 3,
 'parallelism': 5,
 'cross_validation_stratergy': 'Time-based K-fold(with overlap)',
 'num_of_folds': 6,
 'split_ratio': 0,
 'stratified': True}

In [132]:
def fit_and_predict_with_tuning(X, y, selected_models, parameter):
    
    if parameter["cross_validation_stratergy"] == "Time-based K-fold(with overlap)":
        cv = TimeSeriesSplit(n_splits=parameter["num_of_folds"])
    elif parameter["stratified"]:
        cv = StratifiedKFold(n_splits=parameter["num_of_folds"], shuffle=parameter["shuffle_grid"], random_state=parameter["random_state"])
    else:
        cv = KFold(n_splits=parameter["num_of_folds"], shuffle=parameter["shuffle_grid"], random_state=parameter["random_state"])

    results = {}
    for model_name, model_config in selected_models.items():
        print(f"\nRunning GridSearchCV for {model_name}...")
        if model_name == "RandomForestRegressor":
            model = RandomForestRegressor(random_state=parameter["random_state"])
            param_grid = {
                "n_estimators": list(range(model_config["min_trees"], model_config["max_trees"] + 1)),
                "max_depth": list(range(model_config["min_depth"], model_config["max_depth"] + 1)),
                "min_samples_leaf": list(range(
                    model_config["min_samples_per_leaf_min_value"],
                    model_config["min_samples_per_leaf_max_value"] + 1
                ))
            }
        
        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grid,
            cv=cv,
            scoring='neg_mean_squared_error' if "Regressor" in model_name else 'accuracy',
            n_jobs=parameter["parallelism"]
        )
        grid_search.fit(X, y)

        best_model = grid_search.best_estimator_
        results[model_name] = {
            "best_params": grid_search.best_params_,
            "best_score": grid_search.best_score_,
        }

        print(f"Best parameters for {model_name}: {grid_search.best_params_}")
        print(f"Best score for {model_name}: {grid_search.best_score_}")

       
        if "Regressor" in model_name:
            predictions = best_model.predict(X)
            mse = mean_squared_error(y, predictions)
            r2 = r2_score(y, predictions)
            print(f"Mean Squared Error (MSE) for {model_name}: {mse}")
            print(f"R-squared (R2) for {model_name}: {r2}")
        

    return results


In [134]:
testing =fit_and_predict_with_tuning(reduction,y,selected_models,parameter)


Running GridSearchCV for RandomForestRegressor...
Best parameters for RandomForestRegressor: {'max_depth': 20, 'min_samples_leaf': 5, 'n_estimators': 10}
Best score for RandomForestRegressor: -0.2026151136739501
Mean Squared Error (MSE) for RandomForestRegressor: 0.019539404413639206
R-squared (R2) for RandomForestRegressor: 0.9662258623129454


In [136]:
def pipeline(rtf_file_path, output_json_path, csv_path):
    rtf_file = convert_rtf_json(rtf_file_path,output_json_path)
    with open(output_json_path,'r') as file:
         data = json.load(file)
    df = load_data(csv_path)
    target_predictive_type = extracting_target_predictive_type(data)
    y = df[target_predictive_type['target']]
    X = df.drop(target_predictive_type['target'], axis=1)
    encode_feature = encode_categorical_features(X)
    mis = handle_missing_values(df,data)
    reduction =feature_reduction(X, y, data, target_predictive_type,encode_feature)
    selected_models = select_models(data, target_predictive_type)
    print("\nFinal Selected Models with Parameters:")
    print("=" * 50)
    for model_name, model_config in selected_models.items():
        print(f"\n{model_name}:")
        for param_name, param_value in model_config.items():
            if param_name != "model_name":
                print(f"- {param_name}: {param_value}")
    parameter = parse_hyper(data)
    predict =fit_and_predict_with_tuning(reduction,y,selected_models,parameter)
    
    return predict
    

In [137]:
pipe = pipeline("algoparams_from_ui.json.rtf","algoparams_from_ui.json","iris.csv",)